# Blake Conrad
HW1 Advanced Simulations

# Libraries

In [2]:
import pandas as pd
import numpy as np

# P1

# Simulate M/M/1 Queue

In [103]:
df = pd.DataFrame({'RecordType':[],
                   'Time':[],
                    'NextArrival':[],
                    'NextDeparture':[],
                    'Utilization':[],
                    'QueueLength':[],
                    'TotalWaitingTimeInLine':[],
                    'TotalWaitingTimeSystem':[]})


# Subroutines

In [104]:
def getArrival(time, rate=4.0):
    return np.random.exponential(scale=rate) + time
#getArrival(0.0)

def getDeparture(time, rate=5.0):
    return np.random.exponential(scale=rate) + time
#getDeparture(0.0)

def addRow(tmp, insert_dict, index):
    keys=list(insert_dict.keys())
    assert('recordType' in keys, "recordType not in dictionary for insertion.")
    assert('time' in keys, "time not in dictionary for insertion.")
    assert('nextArrival' in keys, "nextArrival not in dictionary for insertion.")
    assert('nextDeparture' in keys, "nextDeparture not in dictionary for insertion.")
    assert('utilization' in keys, "utilization not in dictionary for insertion.")
    assert('queueLength' in keys, "queueLength not in dictionary for insertion.")
    assert('totalTimeInLine' in keys, "totalTimeInLine not in dictionary for insertion.")
    assert('totalTimeInSystem' in keys, "totalTimeInSystem not in dictionary for insertion.")
    tmp.loc[index]=[insert_dict['recordType'],
                    insert_dict['time'],
                    insert_dict['nextArrival'],
                    insert_dict['nextDeparture'],
                    insert_dict['utilization'],
                    insert_dict['queueLength'],
                    insert_dict['totalTimeInLine'],
                    insert_dict['totalTimeInSystem']]
    index+=1
    return tmp, index

#addRow(df, 0)
df

<ipython-input-104-b5abec5269a4>:11: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert('recordType' in keys, "recordType not in dictionary for insertion.")
<ipython-input-104-b5abec5269a4>:12: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert('time' in keys, "time not in dictionary for insertion.")
<ipython-input-104-b5abec5269a4>:13: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert('nextArrival' in keys, "nextArrival not in dictionary for insertion.")
<ipython-input-104-b5abec5269a4>:14: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert('nextDeparture' in keys, "nextDeparture not in dictionary for insertion.")
<ipython-input-104-b5abec5269a4>:15: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert('utilization' in keys, "utilization not in dictionary for insertion.")
<ipython-input-104-b5abec5269a4>:16: SyntaxWarning: assertion is always true, perh

,RecordType,Time,NextArrival,NextDeparture,Utilization,QueueLength,TotalWaitingTimeInLine,TotalWaitingTimeSystem


# Init

In [79]:
M=1000000
time=0.0
oldTime=0.0
nextDeparture=M
nextArrival=getArrival(time)
utilization=0
queueLength=0
totalTimeInLine=0
totalTimeInSystem=0
index_count=0

insertDict={'recordType':'A','oldTime':oldTime, 'time':time, 
             'nextArrival':nextArrival,
             'nextDeparture':nextDeparture,
             'utilization':0, 
             'queueLength':0, 
             'totalTimeInLine':0, 
             'totalTimeInSystem':0}
df, idx=addRow(df, insertDict ,index_count)
df.head()

,RecordType,Time,NextArrival,NextDeparture,Utilization,QueueLength,TotalWaitingTimeInLine,TotalWaitingTimeSystem
0,A,0.0,8.399345,1000000.0,0.0,0.0,0.0,0.0


# Main

In [80]:
while time <= 160:
    # Update time
    oldTime=time
    
    # Template for next record in db
    insertDict={'recordType':'','time':0, 
             'nextArrival':0,
             'nextDeparture':0,
             'utilization':0, 
             'queueLength':0, 
             'totalTimeInLine':0, 
             'totalTimeInSystem':0}
    
    # Handle arrival record
    if nextArrival < nextDeparture:
        time=getArrival(time)
        delta=time-oldTime
        totalTimeInLine=delta*queueLength + totalTimeInLine
        totalTimeInSystem=(utilization+queueLength)*delta + totalTimeInSystem
        if utilization==0:
            utilization=1
            nextDeparture=time+getDeparture(time)
        else:
            queueLength+=1
        
        # Insert information
        nextArrival = time + getArrival(time)
        insertDict['recordType']='A'
        insertDict['time']=time
        insertDict['nextArrival']=nextArrival
        insertDict['nextDeparture']=nextDeparture
        insertDict['utilization']=utilization
        insertDict['queueLength']=queueLength
        insertDict['totalTimeInLine']=totalTimeInLine
        insertDict['totalTimeInSystem']=totalTimeInSystem

        # Add arrival record
        df, idx = addRow(df,insertDict, idx)
    
    # Handle departure record
    else:
        time=nextDeparture
        if queueLength >= 1:
            queueLength -= 1
            nextDeparture = time + getDeparture(time)
        else:
            utilization=0
            nextDeparture=M
            oldTime=time
            
        # Insert information
        insertDict['recordType']='D'
        insertDict['time']=time
        insertDict['nextArrival']=nextArrival
        insertDict['nextDeparture']=nextDeparture
        insertDict['utilization']=utilization
        insertDict['queueLength']=queueLength
        insertDict['totalTimeInLine']=totalTimeInLine
        insertDict['totalTimeInSystem']=totalTimeInSystem
        
        # Add departure record
        df, idx = addRow(df,insertDict, idx)
df

In [81]:
df

,RecordType,Time,NextArrival,NextDeparture,Utilization,QueueLength,TotalWaitingTimeInLine,TotalWaitingTimeSystem
0,A,0.000000,8.399345,1000000.000000,0.0,0.0,0.000000,0.000000
1,A,7.370542,16.976837,21.660340,1.0,0.0,0.000000,0.000000
2,A,9.590865,24.784540,21.660340,1.0,1.0,0.000000,2.220322
3,D,21.660340,24.784540,45.687943,1.0,0.0,0.000000,2.220322
4,A,23.331678,57.909936,45.687943,1.0,1.0,0.000000,3.891660
5,D,45.687943,57.909936,97.853263,1.0,0.0,0.000000,3.891660
6,A,46.299235,92.660432,97.853263,1.0,1.0,0.000000,4.502952
7,A,49.095376,99.176781,97.853263,1.0,2.0,2.796141,10.095234
8,D,97.853263,99.176781,201.003482,1.0,1.0,2.796141,10.095234
9,A,99.289623,200.791152,201.003482,1.0,2.0,4.232501,12.967953


# Analysis
- Average time in queue: <strong>About 30 minutes.</strong>
- Average time in system: <strong>About 30 minutes.</strong>
- Average number in queue: <strong>About 1 hour.</strong>
- Average utilization: <strong>About 100%, or 1.</strong>

In [82]:
# Average time in queue -- About a half an hour.
21.045225/38.187039

0.5511091079881841

In [100]:
#Average time in system -- Little over a half an hour.
df[["Utilization", "QueueLength"]].sum(axis=1).sum()/38.187039

0.6284854921587401

In [101]:
# Average number in queue -- Little over an hour in the queue.
df[["Utilization", "QueueLength"]].sum(axis=1).sum()/21.045225

1.1404012073997785

In [99]:
# Average utilization -- About 100% utilization for this 1 server queue design.
df.Utilization.mean()

0.9166666666666666

In [32]:
def guessRain():
    return np.random.choice(["rain", "no_rain"],p=[0.2,0.8])
def guessType():
    return np.random.choice(["shower", "rain", "downpour"],p=[0.75,0.2, 0.05])

P={"rain":{"shower":np.random.uniform(0.05, 0.6),
           "rain":np.random.uniform(0.4, 1.2),
           "downpour":np.random.uniform(12,60)},
   "no_rain":0}

# P2

In [45]:
df = pd.DataFrame({"rep":[],
                    "year":[],
                    "finalHeight":[]})
def addRow(tmp, insert_dict, index):
    index+=1
    tmp.loc[index]=[insert_dict['rep'],
                    insert_dict['year'],
                    insert_dict['finalHeight']]
    return tmp, index
idx=0
for rep in range(30):
    print(rep)
    for year in range(100):
        normalHeight=13296
        currentHeight=normalHeight
        for day in range(365):
            
            # Simple gate checks
            if currentHeight > normalHeight + 60:
                currentHeight -= 0.5
            else:
                currentHeight -= 0.25
            
            # Day begins
            rain_maybe = guessRain()
            if rain_maybe == "rain":
                rain_type = guessType()
                rainIncoming=rain_type
                rain_incoming = P["rain"][rain_type]
            else:
                rainIncoming="no_rain"
                rain_incoming = P["no_rain"]
            
            # Run calculations
            if rain_incoming < 0.4:
                rain_added = rain_incoming
            elif rain_incoming >= 0.4 and rain_incoming < 1:
                rain_added = 2*rain_incoming
            elif rain_incoming >= 1:
                rain_added = 5*rain_incoming
                
            currentHeight += rain_added
            
        # Insert data
        df, idx = addRow(df, {"rep":rep,"year":year,"finalHeight":currentHeight}, idx)
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


,rep,year,finalHeight
1,0.0,0.0,14024.554043
2,0.0,1.0,13870.883381
3,0.0,2.0,14212.214659
4,0.0,3.0,14042.571163
5,0.0,4.0,14209.018276
6,0.0,5.0,13529.607915
7,0.0,6.0,14530.922210
8,0.0,7.0,14385.096629
9,0.0,8.0,14059.440927
10,0.0,9.0,13929.334354


In [56]:
# Average final height each rep
max_every_hundred_years=df.groupby(by="rep").finalHeight.max()
print(max_every_hundred_years)
average_max_height_every_hundred_years = max_every_hundred_years.mean()
print(average_max_height_every_hundred_years)

rep
0.0     14530.922210
1.0     14551.560862
2.0     14692.853634
3.0     14875.635067
4.0     14383.611266
5.0     14540.214728
6.0     14558.662453
7.0     14868.843264
8.0     14721.289255
9.0     14710.280551
10.0    14566.294168
11.0    14588.219317
12.0    14716.595694
13.0    14729.116928
14.0    14707.137447
15.0    14868.797637
16.0    14858.800747
17.0    15059.697780
18.0    14704.252332
19.0    14896.482205
20.0    14717.532319
21.0    14887.064756
22.0    15053.337773
23.0    15039.138569
24.0    14561.538575
25.0    14577.721375
26.0    14544.959134
27.0    14703.140557
28.0    14881.054284
29.0    15222.610452
Name: finalHeight, dtype: float64
14743.912177914897


# Analysis
We expect every one hundred years for the height to be no greater than <em>14743.912177914897</em>. Hence, we will recommend the Army Corps of Engineers to build their dam no greater than about 14744 inches, or 1229 feet.

In [57]:
14743/12

1228.5833333333333

# P3

In [101]:
P={"McGuire":np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.127,0.05, .001, 0.094 ,0.728]),
   "Gwynn":np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.202,.059, .015, 0.063 ,0.661])}
team1=[P["McGuire"] for i in range(9)]
team2=[P["Gwynn"] for i in range(9)]
team3=[P["Gwynn"],P["Gwynn"],P["McGuire"],P["McGuire"],P["McGuire"],P["McGuire"],P["Gwynn"],P["Gwynn"],P["Gwynn"]]
len(team1) == len(team2) == len(team3)

True

# McQuire vs. Gwynn

In [129]:
# 1 vs. 2
scores1=[]
for _ in range(100):
    team=1
    team1_outs=0
    team2_outs=0
    team1_score=0
    team2_score=0
    batterUp1=0
    batterUp2=0
    bases=[0,0,0,0]
    inning=1
    while inning != 18:
        if team > 0: #team 1
            #result=team1[batterUp1]
            result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.127,0.05, .001, 0.094 ,0.728])
            if result == "Out":
                team1_outs+=1
                batterUp1+=1
                if batterUp1 > 8:
                    batterUp1=0
                if team1_outs >= 4:
                    team*=-1
                    inning+=1
                    bases=[0,0,0,0]
            else:
                if result == "Single":
                    # Update point criteria
                    if bases[-1] == 1:
                        team1_score+=1
                    # Update positions
                    for i in range(3):
                        bases[i+1]=bases[i]
                    bases[0]=1
                    bases[3]=0
                elif result == "Double":
                    # Update point criteria
                    if bases[-2] == 1:
                        team1_score+=1
                    if bases[-1] == 1:
                        team1_score+=1
                    # Update positions
                    for i in range(2):
                        bases[i+2]=bases[i]
                    bases[0]=0
                    bases[1]=1
                    bases[3]=0
                elif result == "Triple":
                    # Update point criteria
                    if bases[-3] == 1:
                        team1_score+=1
                    if bases[-2] == 1:
                        team1_score+=1
                    if bases[-1] == 1:
                        team1_score+=1
                    bases[0]=0
                    bases[1]=0
                    bases[2]=1
                    bases[3]=0
                elif result == "HomeRun":
                    team1_score+=1
                    team1_score+=sum(bases)
                    bases[0]=0
                    bases[1]=0
                    bases[2]=0
                    bases[3]=0
            #print(result)
            #print(bases)
            #print(team1_score)
            #input("...")

        else: #team 2
            #result=team2[batterUp2]
            result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.202,.059, .015, 0.063 ,0.661])
            if result == "Out":
                team2_outs+=1
                batterUp2+=1
                if batterUp2 > 8:
                    batterUp2=0
                if team2_outs >= 4:
                    team*=-1
                    inning+=1
                    bases=[0,0,0,0]
            else:
                if result == "Single":
                    # Update point criteria
                    if bases[-1] == 1:
                        team2_score+=1
                    # Update positions
                    for i in range(3):
                        bases[i+1]=bases[i]
                    bases[0]=1
                    bases[3]=0
                elif result == "Double":
                    # Update point criteria
                    if bases[-2] == 1:
                        team2_score+=1
                    if bases[-1] == 1:
                        team2_score+=1
                    # Update positions
                    for i in range(2):
                        bases[i+2]=bases[i]
                    bases[0]=0
                    bases[1]=1
                    bases[3]=0
                elif result == "Triple":
                    # Update point criteria
                    if bases[-3] == 1:
                        team2_score+=1
                    if bases[-2] == 1:
                        team2_score+=1
                    if bases[-1] == 1:
                        team2_score+=1
                    bases[0]=0
                    bases[1]=0
                    bases[2]=1
                    bases[3]=0
                elif result == "HomeRun":
                    team2_score+=1
                    team2_score+=sum(bases)
                    bases[0]=0
                    bases[1]=0
                    bases[2]=0
                    bases[3]=0
            #print(result)
            #print(bases)
            #print(team2_score)
            #input("...")
        scores1.append((team1_score, team2_score))
print("Averages Points Scored By McQuire {}\nAverage Points Scored By Gwynn {}\n".format(sum(list(map(lambda x:x[0], scores)))/len(scores), sum(list(map(lambda x:x[1], scores)))/len(scores)))


Averages Points Scored By McQuire 1.1658062556938962
Average Points Scored By Gwynn 1.032796841785606



# McQuire vs. Hybrid

In [131]:
# 1 vs 3
scores2=[]
for _ in range(100):
    team=1
    team1_outs=0
    team2_outs=0
    team1_score=0
    team2_score=0
    batterUp1=0
    batterUp2=0
    bases=[0,0,0,0]
    inning=1
    while inning != 18:
        if team > 0: #team 1
            #result=team1[batterUp1]
            result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.127,0.05, .001, 0.094 ,0.728])
            if result == "Out":
                team1_outs+=1
                batterUp1+=1
                if batterUp1 > 8:
                    batterUp1=0
                if team1_outs >= 4:
                    team*=-1
                    inning+=1
                    bases=[0,0,0,0]
            else:
                if result == "Single":
                    # Update point criteria
                    if bases[-1] == 1:
                        team1_score+=1
                    # Update positions
                    for i in range(3):
                        bases[i+1]=bases[i]
                    bases[0]=1
                    bases[3]=0
                elif result == "Double":
                    # Update point criteria
                    if bases[-2] == 1:
                        team1_score+=1
                    if bases[-1] == 1:
                        team1_score+=1
                    # Update positions
                    for i in range(2):
                        bases[i+2]=bases[i]
                    bases[0]=0
                    bases[1]=1
                    bases[3]=0
                elif result == "Triple":
                    # Update point criteria
                    if bases[-3] == 1:
                        team1_score+=1
                    if bases[-2] == 1:
                        team1_score+=1
                    if bases[-1] == 1:
                        team1_score+=1
                    bases[0]=0
                    bases[1]=0
                    bases[2]=1
                    bases[3]=0
                elif result == "HomeRun":
                    team1_score+=1
                    team1_score+=sum(bases)
                    bases[0]=0
                    bases[1]=0
                    bases[2]=0
                    bases[3]=0
            #print(result)
            #print(bases)
            #print(team1_score)
            #input("...")

        else: #team 2
            #result=team2[batterUp2]
            if batterUp2 >= 0 and batterUp2 <= 2:
                result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.202,.059, .015, 0.063 ,0.661])
            elif batterUp2 >2 and batterUp2 <= 6:
                result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.127,0.05, .001, 0.094 ,0.728])
            else:
                result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.202,.059, .015, 0.063 ,0.661])
            if result == "Out":
                team2_outs+=1
                batterUp2+=1
                if batterUp2 > 8:
                    batterUp2=0
                if team2_outs >= 4:
                    team*=-1
                    inning+=1
                    bases=[0,0,0,0]
            else:
                if result == "Single":
                    # Update point criteria
                    if bases[-1] == 1:
                        team2_score+=1
                    # Update positions
                    for i in range(3):
                        bases[i+1]=bases[i]
                    bases[0]=1
                    bases[3]=0
                elif result == "Double":
                    # Update point criteria
                    if bases[-2] == 1:
                        team2_score+=1
                    if bases[-1] == 1:
                        team2_score+=1
                    # Update positions
                    for i in range(2):
                        bases[i+2]=bases[i]
                    bases[0]=0
                    bases[1]=1
                    bases[3]=0
                elif result == "Triple":
                    # Update point criteria
                    if bases[-3] == 1:
                        team2_score+=1
                    if bases[-2] == 1:
                        team2_score+=1
                    if bases[-1] == 1:
                        team2_score+=1
                    bases[0]=0
                    bases[1]=0
                    bases[2]=1
                    bases[3]=0
                elif result == "HomeRun":
                    team2_score+=1
                    team2_score+=sum(bases)
                    bases[0]=0
                    bases[1]=0
                    bases[2]=0
                    bases[3]=0
            #print(result)
            #print(bases)
            #print(team2_score)
            #input("...")
        scores2.append((team1_score, team2_score))
print("Averages Points Scored By McQuire {}\nAverage Points Scored By Hybrid {}\n".format(sum(list(map(lambda x:x[0], scores2)))/len(scores2), sum(list(map(lambda x:x[1], scores2)))/len(scores2)))


Averages Points Scored By McQuire 1.1785164666051093
Average Points Scored By Hybrid 1.2449984610649432



# Gwynn vs. Hybrid

In [130]:

# 2 vs. 3
scores3=[]
for _ in range(100):
    team=1
    team1_outs=0
    team2_outs=0
    team1_score=0
    team2_score=0
    batterUp1=0
    batterUp2=0
    bases=[0,0,0,0]
    inning=1
    while inning != 18:
        if team > 0: #team 1
            #result=team1[batterUp1]
            result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.202,.059, .015, 0.063 ,0.661])
            if result == "Out":
                team1_outs+=1
                batterUp1+=1
                if batterUp1 > 8:
                    batterUp1=0
                if team1_outs >= 4:
                    team*=-1
                    inning+=1
                    bases=[0,0,0,0]
            else:
                if result == "Single":
                    # Update point criteria
                    if bases[-1] == 1:
                        team1_score+=1
                    # Update positions
                    for i in range(3):
                        bases[i+1]=bases[i]
                    bases[0]=1
                    bases[3]=0
                elif result == "Double":
                    # Update point criteria
                    if bases[-2] == 1:
                        team1_score+=1
                    if bases[-1] == 1:
                        team1_score+=1
                    # Update positions
                    for i in range(2):
                        bases[i+2]=bases[i]
                    bases[0]=0
                    bases[1]=1
                    bases[3]=0
                elif result == "Triple":
                    # Update point criteria
                    if bases[-3] == 1:
                        team1_score+=1
                    if bases[-2] == 1:
                        team1_score+=1
                    if bases[-1] == 1:
                        team1_score+=1
                    bases[0]=0
                    bases[1]=0
                    bases[2]=1
                    bases[3]=0
                elif result == "HomeRun":
                    team1_score+=1
                    team1_score+=sum(bases)
                    bases[0]=0
                    bases[1]=0
                    bases[2]=0
                    bases[3]=0
            #print(result)
            #print(bases)
            #print(team1_score)
            #input("...")

        else: #team 2
            #result=team2[batterUp2]
            if batterUp2 >= 0 and batterUp2 <= 2:
                result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.202,.059, .015, 0.063 ,0.661])
            elif batterUp2 >2 and batterUp2 <= 6:
                result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.127,0.05, .001, 0.094 ,0.728])
            else:
                result=np.random.choice(["Single", "Double", "Triple", "HomeRun", "Out"],p=[.202,.059, .015, 0.063 ,0.661])
            if result == "Out":
                team2_outs+=1
                batterUp2+=1
                if batterUp2 > 8:
                    batterUp2=0
                if team2_outs >= 4:
                    team*=-1
                    inning+=1
                    bases=[0,0,0,0]
            else:
                if result == "Single":
                    # Update point criteria
                    if bases[-1] == 1:
                        team2_score+=1
                    # Update positions
                    for i in range(3):
                        bases[i+1]=bases[i]
                    bases[0]=1
                    bases[3]=0
                elif result == "Double":
                    # Update point criteria
                    if bases[-2] == 1:
                        team2_score+=1
                    if bases[-1] == 1:
                        team2_score+=1
                    # Update positions
                    for i in range(2):
                        bases[i+2]=bases[i]
                    bases[0]=0
                    bases[1]=1
                    bases[3]=0
                elif result == "Triple":
                    # Update point criteria
                    if bases[-3] == 1:
                        team2_score+=1
                    if bases[-2] == 1:
                        team2_score+=1
                    if bases[-1] == 1:
                        team2_score+=1
                    bases[0]=0
                    bases[1]=0
                    bases[2]=1
                    bases[3]=0
                elif result == "HomeRun":
                    team2_score+=1
                    team2_score+=sum(bases)
                    bases[0]=0
                    bases[1]=0
                    bases[2]=0
                    bases[3]=0
            #print(result)
            #print(bases)
            #print(team2_score)
            #input("...")
        scores3.append((team1_score, team2_score))
        
print("Averages Points Scored By Gwynn {}\nAverage Points Scored By Hybrid {}\n".format(sum(list(map(lambda x:x[0], scores3)))/len(scores3), sum(list(map(lambda x:x[1], scores3)))/len(scores3)))


Averages Points Scored By Gwynn 1.2669748627564288
Average Points Scored By Hybrid 1.360011557353366



# Analysis

Averages Points Scored By McQuire 1.1658062556938962
Average Points Scored By Gwynn 1.032796841785606

Averages Points Scored By McQuire 1.1785164666051093
Average Points Scored By Hybrid 1.2449984610649432

Averages Points Scored By Gwynn 1.2669748627564288
Average Points Scored By Hybrid 1.360011557353366

Both McQuire and Gwynn are better together than separate (Great complementary team mates)! But McQuire is better than Gwynn in isolation.